In [7]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [8]:
def split_time(df):
    sub = df['patient_id'].split('-')
    try:
        time = int(sub[2])
        return time
    except IndexError:
        return 0

In [9]:
def remove_time(df):
    patient = df['patient_id'].split('-')
    return '{}-{}'.format(patient[0], patient[1])

In [10]:
def remove_day(x):
    if isinstance(x, str):
        x = x.replace('day ', '')
    return x

In [11]:
add_info = pd.read_stata('C:/Users/lzoeckler/Desktop/additional_info.dta')
add_info.head()

,priority_level,sample_id,pa_id,boxnumber,position,day0_enrollee,days_since_tx,initial_sample,got_PQ,age,sex,fever48_r,enrolled_from,returned_with_fever,when_returned_with_fever,retreated,when_retreated,comments
0,1.0,PA-001,PA-001,1,7,1.0,0.0,1.0,1.0,2,Female,1.0,Health facility,NaN,,NaN,,
1,2.0,PA-001-3,PA-001,1,44,1.0,3.0,NaN,1.0,2,Female,1.0,Health facility,NaN,,NaN,,
2,2.0,PA-001-7,PA-001,2,48,1.0,9.0,NaN,1.0,2,Female,1.0,Health facility,NaN,,NaN,,
3,2.0,PA-001-14,PA-001,4,60,1.0,17.0,NaN,1.0,2,Female,1.0,Health facility,NaN,,NaN,,
4,2.0,PA-001-21,PA-001,6,16,1.0,23.0,NaN,1.0,2,Female,1.0,Health facility,NaN,,NaN,,


In [139]:
add_info = add_info.applymap(lambda x: x.lower() if isinstance(x, str) else x)
add_info.rename(columns={'sample_id': 'patient_id'}, inplace=True)
add_info.drop(['pa_id', 'priority_level', 'boxnumber', 'position', 'comments'], axis=1, inplace=True)
add_info['time_point_days'] = add_info.apply(split_time, axis=1)
add_info['patient_id'] = add_info.apply(remove_time, axis=1)
add_info.drop_duplicates(subset=['patient_id', 'time_point_days'], inplace=True, keep='last')
add_info['when_returned_with_fever'] = add_info['when_returned_with_fever'].apply(remove_day)
add_info['when_retreated'] = add_info['when_retreated'].apply(remove_day)
add_info.head()

,patient_id,day0_enrollee,days_since_tx,initial_sample,got_PQ,age,sex,fever48_r,enrolled_from,returned_with_fever,when_returned_with_fever,retreated,when_retreated,time_point_days
0,pa-001,1.0,0.0,1.0,1.0,2,female,1.0,health facility,NaN,,NaN,,0
1,pa-001,1.0,3.0,NaN,1.0,2,female,1.0,health facility,NaN,,NaN,,3
2,pa-001,1.0,9.0,NaN,1.0,2,female,1.0,health facility,NaN,,NaN,,7
3,pa-001,1.0,17.0,NaN,1.0,2,female,1.0,health facility,NaN,,NaN,,14
4,pa-001,1.0,23.0,NaN,1.0,2,female,1.0,health facility,NaN,,NaN,,21


In [140]:
time_df = super_final.copy(deep=True)
time_df['time_point_days'] = time_df.apply(split_time, axis=1)
time_df['patient_id'] = time_df.apply(remove_time, axis=1)
time_df = time_df.merge(add_info, how='inner', on=['patient_id', 'time_point_days'])
time_df.sort_values(['patient_id', 'time_point_days'], inplace=True)
time_df.set_index(['patient_id', 'time_point_days'], inplace=True)
time_df.head()

errors HRP2_pg_ml HRP2_pg_ml_dilution  \
patient_id time_point_days                                         
pa-001     3                  NaN  318702.59                2500   
           7                  NaN   73737.79                2500   
           14                 NaN    4138.79                  50   
           21                 NaN    1377.18                  50   
           28                 NaN    2024.41                  50   

                           HRP2_pg_ml_well HRP2_pg_ml_max_dilution  \
patient_id time_point_days                                           
pa-001     3                            e3               312500000   
           7                            c5                  125000   
           14                           d8                      50   
           21                          d12                      50   
           28                           h2                      50   

                           LDH_Pan_pg_ml LDH_Pan_pg_ml_dilution  \
patient_id time_point_days                                        
pa-001     3                      142.30                      1   
           7                      179.34                      1   
           14                    < 14.41                      1   
           21                      38.94                      1   
           28                      29.86                      1   

                           LDH_Pan_pg_ml_well LDH_Pan_pg_ml_max_dilution  \
patient_id time_point_days                                                 
pa-001     3                               g6                  312500000   
           7                              e12                     125000   
           14                              c8                         50   
           21                             c12                         50   
           28                              g2                         50   

                           LDH_Pv_pg_ml  ... initial_sample got_PQ age  \
patient_id time_point_days               ...                             
pa-001     3                      15.35  ...            NaN    1.0   2   
           7                      10.31  ...            NaN    1.0   2   
           14                      2.76  ...            NaN    1.0   2   
           21                     12.21  ...            NaN    1.0   2   
           28                     13.90  ...            NaN    1.0   2   

                               sex fever48_r    enrolled_from  \
patient_id time_point_days                                      
pa-001     3                female       1.0  health facility   
           7                female       1.0  health facility   
           14               female       1.0  health facility   
           21               female       1.0  health facility   
           28               female       1.0  health facility   

                           returned_with_fever  when_returned_with_fever  \
patient_id time_point_days                                                 
pa-001     3                               NaN                             
           7                               NaN                             
           14                              NaN                             
           21                              NaN                             
           28                              NaN                             

                            retreated  when_retreated  
patient_id time_point_days                             
pa-001     3                      NaN                  
           7                      NaN                  
           14                     NaN                  
           21                     NaN                  
           28                     NaN                  

[5 rows x 29 columns]

In [141]:
time_df.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/final_dilutions.csv')

In [142]:
partial_format = samples_data.copy(deep=True)
partial_format['time_point_days'] = partial_format.apply(split_time, axis=1)
partial_format['patient_id'] = partial_format.apply(remove_time, axis=1)
partial_format = partial_format.merge(add_info, how='left', on=['patient_id', 'time_point_days'])
partial_format.sort_values(['patient_id', 'time_point_days'], inplace=True)
partial_format.set_index(['patient_id', 'time_point_days'], inplace=True)
partial_format.head()

well error     HRP2_pg_ml    LDH_Pan_pg_ml  \
patient_id time_point_days                                              
pa-001     3                 g6   NaN         145.95           142.30   
           3                 f3   NaN      397468.45     < 1801250.00   
           3                 e3   NaN      318702.59         74360.14   
           3                 h3   NaN  1537195777.01  119019730410.15   
           3                 h6   NaN       10273.61          2535.11   

                              LDH_Pv_pg_ml        CRP_ng_ml concentration  \
patient_id time_point_days                                                  
pa-001     3                         15.35          5678.58             1   
           3                   < 291250.00     < 3342500.00        125000   
           3                      10385.14       < 66850.00          2500   
           3                16185275422.09  286153421264.94     312500000   
           3                        733.44         49491.76            50   

                            day0_enrollee  days_since_tx  initial_sample  \
patient_id time_point_days                                                 
pa-001     3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   

                            got_PQ  age     sex  fever48_r    enrolled_from  \
patient_id time_point_days                                                    
pa-001     3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   

                            returned_with_fever when_returned_with_fever  \
patient_id time_point_days                                                 
pa-001     3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            

                            retreated when_retreated  
patient_id time_point_days                            
pa-001     3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN

In [143]:
partial_format.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/partially_formatted.csv')